In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/CarloSgaravatti/Recsys_Challenge_2023.git

Cloning into 'Recsys_Challenge_2023'...
remote: Enumerating objects: 1990, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 1990 (delta 68), reused 4 (delta 4), pack-reused 1882
Receiving objects: 100% (1990/1990), 270.31 MiB | 18.62 MiB/s, done.
Resolving deltas: 100% (1520/1520), done.


In [2]:
! pip install PyGithub requests

  Obtaining dependency information for PyGithub from https://files.pythonhosted.org/packages/be/04/810d131be173cba445d3658a45512b2b2b3d0960d52c4a300d6ec5e00f52/PyGithub-2.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 11.8 MB/s eta 0:00:00


In [3]:
! cd /kaggle/working/Recsys_Challenge_2023 && python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorization_Cython_Epoch.c:1109:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorization_Cython_Epoch.c: In function ‘__pyx_pf_32MatrixFactorization

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import random
import tqdm
import gc
import time
import os
import shutil
import optuna
import scipy.sparse as sps

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv
/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv
/kaggle/input/training-tuned-models/slim_bpr_xgboost_train_validation.zip
/kaggle/input/training-tuned-models/__results__.html
/kaggle/input/training-tuned-models/ease_r_all.zip
/kaggle/input/training-tuned-models/slim_bpr_all.zip
/kaggle/input/training-tuned-models/slim_elastic_train_validation.zip
/kaggle/input/training-tuned-models/ease_r_train_validation.zip
/kaggle/input/training-tuned-models/__notebook__.ipynb
/kaggle/input/training-tuned-models/ials_xgboost_train_validation.zip
/kaggle/input/training-tuned-models/ials_train_validation.zip
/kaggle/input/training-tuned-models/slim_elastic_xgboost_train_validation.zip
/kaggle/input/training-tuned-models/ease_r_xgboost_train_validation.zip
/kaggle/input/training-tuned-models/__output__.json
/kaggle/input/training-tuned-models/slim_bpr_train_validation.zip
/kaggle/input/training-tuned

In [5]:
%cd /kaggle/working/Recsys_Challenge_2023

/kaggle/working/Recsys_Challenge_2023


In [6]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender, SLIMElasticNetRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender, PureSVDItemRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.SLIM.SLIM_BPR_Python import SLIM_BPR_Python
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender
from Recommenders.BaseRecommender import BaseRecommender

from Recommenders.Hybrid.LinearCombinationRecommender import LinearCombinationRecommender
from Data_manager.CrossValidationSplitter import CrossValidationSplitter

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

# Configurations

In [7]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'Recsys_Challenge_2023':
        repo = r
        print('Repository found')

Repository found


In [8]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())

In [9]:
config = {
    'tune_parameters': True,
    'database_path': '/kaggle/working/tuning_hybrid.db',
    'copy_prev_tuning_db': True,
    'copy_prev_best_params': True,
    'save_github': True
}

# Tuning

In [10]:
train_data = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv')
user_ids_test = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv')

In [11]:
URM_all = sps.csr_matrix((train_data.data.values,
                          ((train_data.row - 1).values, (train_data.col - 1).values)),
                        shape=(train_data.row.max(), train_data.col.max()))
URM_all

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [12]:
URM_train, URM_test_complete = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)

In [13]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_item.json', 'r') as params_file:
    params_item_knn = json.load(params_file)
params_item_knn

{'topK': 14,
 'shrink': 25,
 'similarity': 'tversky',
 'tversky_alpha': 0.18163095712123836,
 'tversky_beta': 1.8529812241698465}

In [14]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_RP3.json', 'r') as params_file:
    params_rp3 = json.load(params_file)
params_rp3

{'alpha': 0.3408516511081212, 'beta': 0.1447198697205665, 'topK': 32}

In [15]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_alpha.json', 'r') as params_file:
    params_p3alpha = json.load(params_file)
params_p3alpha

{'normalize_similarity': 'True',
 'implicit': 'True',
 'alpha': 0.29354071531607107,
 'topK': 49}

In [16]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_hybrid_merging.json', 'r') as params_file:
    params_merging = json.load(params_file)
params_merging

{'alpha': 0.3375591188839621, 'topK': 86}

In [17]:
shutil.copyfile(
    '/kaggle/input/training-tuned-models/ials_train_validation.zip', 
    '/kaggle/working/ials_train_validation.zip', 
)
shutil.copyfile(
    '/kaggle/input/training-tuned-models/ials_train_validation.zip', 
    '/kaggle/working/ials_all.zip', 
)

'/kaggle/working/ials_all.zip'

In [18]:
recommender_item = ItemKNNCFRecommender(URM_train)
recommender_item.fit(**params_item_knn)

recommender_rp3 = RP3betaRecommender(URM_train)
recommender_rp3.fit(**params_rp3)

recommender_p3a = P3alphaRecommender(URM_train)
recommender_p3a.fit(**params_p3alpha)

recommender_slim = SLIMElasticNetRecommender(URM_train)
recommender_slim.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'slim_elastic_train_validation.zip')

recommender_ials = IALSRecommender(URM_train)
recommender_ials.load_model('/kaggle/working/', 'ials_train_validation.zip')

hybrid_similarity = ItemKNNSimilarityHybridRecommender(URM_train, recommender_item.W_sparse, recommender_rp3.W_sparse)
hybrid_similarity.fit(**params_merging)

ItemKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 22347 (100.0%), 4317.79 column/sec. Elapsed time 5.18 sec
RP3betaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
RP3betaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 2290.72 column/sec. Elapsed time 9.76 sec
P3alphaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
P3alphaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
P3alphaRecommender: Similarity column 22347 (100.0%), 2742.32 column/sec. Elapsed time 8.15 sec
SLIMElasticNetRecommender: URM Detected 627 ( 4.8%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 247 ( 1.1%) items with no interactions.
SLIMElasticNetRecommender: Loading model from file '/kaggle/working/Recsys_Challenge_2023/models/slim_elastic_train_validation.zip

In [19]:
class PipelineRecommender(BaseRecommender):
    def __init__(self, URM_train, recommenders, cutoffs):
        super(PipelineRecommender, self).__init__(URM_train)
        self.recommenders = recommenders
        self.cutoffs = cutoffs
        
    def recommend(self, user_ids, cutoff=10, remove_seen_flag=True, remove_top_pop_flag=False, return_scores=True, remove_custom_items_flag=True):
        
        recommendations = []
        scores = np.empty(shape=(0, self.URM_train.shape[1]))
        item_ids = np.arange(self.URM_train.shape[1])
        
        for user_id in user_ids:
            current_recommendations = item_ids.copy()
            for recommender, cut in zip(self.recommenders, self.cutoffs):
                current_recommendations = recommender.recommend(
                    user_id,
                    cutoff=cut,
                    items_to_compute=current_recommendations,
                    remove_seen_flag=remove_seen_flag, 
                    remove_top_pop_flag=remove_top_pop_flag, 
                    return_scores=False, 
                    remove_custom_items_flag=remove_custom_items_flag
                )
                
            recommendations.append(current_recommendations[:cutoff])
            items_score = self.recommenders[-1]._compute_item_score([user_id], current_recommendations[:cutoff])
            scores = np.concatenate([scores, items_score])
            
        if return_scores:
            return np.array(recommendations), scores
        return np.array(recommendations)

In [20]:
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])

print('Merging Models')
result_df, _ = evaluator.evaluateRecommender(hybrid_similarity)
display(result_df)
print('Item KNN')
result_df, _ = evaluator.evaluateRecommender(recommender_item)
display(result_df)
print('RP3 beta')
result_df, _ = evaluator.evaluateRecommender(recommender_rp3)
display(result_df)
print('P3 alpha')
result_df, _ = evaluator.evaluateRecommender(recommender_p3a)
display(result_df)
print('SLIM Elastic')
result_df, _ = evaluator.evaluateRecommender(recommender_slim)
display(result_df)
print('IALS')
result_df, _ = evaluator.evaluateRecommender(recommender_ials)
display(result_df)

EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
Merging Models
EvaluatorHoldout: Processed 10456 (100.0%) in 10.18 sec. Users per second: 1027


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.096595,0.165167,0.138965,0.050024,0.085384,0.274834,0.152525,0.11397,0.519702,0.365722,...,0.802826,0.41723,0.802826,0.083179,10.657938,0.998402,0.240004,0.821046,1.518592,0.309028


Item KNN
EvaluatorHoldout: Processed 10456 (100.0%) in 10.08 sec. Users per second: 1037


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.09451,0.161146,0.135378,0.048719,0.082976,0.267977,0.148683,0.111312,0.511572,0.356662,...,0.802826,0.410703,0.802826,0.066309,10.283111,0.997789,0.191329,0.792171,1.734564,0.30426


RP3 beta
EvaluatorHoldout: Processed 10456 (100.0%) in 10.87 sec. Users per second: 962


...

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.092358,0.15618,0.130929,0.047745,0.081353,0.267656,0.146022,0.108312,0.507269,0.352674,...,0.802826,0.407248,0.802826,0.069984,10.530815,0.998437,0.201933,0.811253,1.356167,0.311674


SLIM Elastic
EvaluatorHoldout: Processed 10456 (100.0%) in 14.14 sec. Users per second: 739


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.097427,0.1648,0.138241,0.051196,0.086161,0.282256,0.154123,0.1143,0.523432,0.375106,...,0.802826,0.420224,0.802826,0.063952,10.572156,0.998771,0.184527,0.814438,1.505137,0.301658


IALS
EvaluatorHoldout: Processed 10456 (100.0%) in 17.11 sec. Users per second: 611


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.08224,0.144123,0.122645,0.041501,0.071584,0.246255,0.131701,0.098458,0.479055,0.316769,...,0.802826,0.384598,0.802826,0.029074,9.74689,0.998532,0.08389,0.750862,1.599865,0.289534


In [21]:
class PipelineRecommender(BaseRecommender):
    def __init__(self, URM_train, recommenders: list[BaseRecommender], cutoffs):
        super(PipelineRecommender, self).__init__(URM_train)
        self.recommenders = recommenders
        self.cutoffs = cutoffs
        
    def recommend(self, user_ids, cutoff=10, remove_seen_flag=True, remove_top_pop_flag=False, return_scores=True, remove_custom_items_flag=True):
        
        recommendations = None
        for recommender_cutoff, recommender in zip(self.cutoffs, self.recommenders):
            recommender_scores = recommender._compute_item_score(user_ids)
            
            for i, user_id in enumerate(user_ids):
                recommender_scores[i, :] = self._remove_seen_on_scores(user_id, recommender_scores[i, :])
            
            if recommendations is None:
                recommendations = np.argsort(recommender_scores, axis=1)[:, -recommender_cutoff:]
                
            else:
                filtered_scores = np.full_like(recommender_scores, -np.inf)
                for i in range(recommendations.shape[0]):
                    row_indices = recommendations[i]
                    filtered_scores[i, row_indices] = recommender_scores[i, row_indices]
                recommendations = np.argsort(filtered_scores, axis=1)[:, -recommender_cutoff:]              
                
        if return_scores:
            return np.flip(recommendations[:, -cutoff:], axis=1), filtered_scores
        return np.flip(recommendations[:, -cutoff:], axis=1)

In [22]:
pipeline_recommender = PipelineRecommender(
    URM_train, 
    recommenders=[recommender_p3a, hybrid_similarity, recommender_slim], 
    cutoffs=[500, 100, 10]
)

result_df, _ = evaluator.evaluateRecommender(pipeline_recommender)
display(result_df)

Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 10456 (100.0%) in 30.52 sec. Users per second: 343


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.096634,0.160608,0.133985,0.051045,0.085245,0.281507,0.152341,0.112284,0.519128,0.373969,...,0.802826,0.416769,0.802826,0.084638,10.945746,0.999078,0.244215,0.843218,1.22117,0.312679


In [23]:
pipeline_recommender = PipelineRecommender(
    URM_train, 
    recommenders=[recommender_p3a, recommender_slim, hybrid_similarity], 
    cutoffs=[500, 100, 10]
)

result_df, _ = evaluator.evaluateRecommender(pipeline_recommender)
display(result_df)

Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 10456 (100.0%) in 31.07 sec. Users per second: 337


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.095161,0.159446,0.133273,0.049685,0.08392,0.273147,0.149759,0.111037,0.510425,0.363393,...,0.802826,0.409782,0.802826,0.094443,10.911388,0.998818,0.272504,0.840571,1.23532,0.315876


In [24]:
pipeline_recommender = PipelineRecommender(
    URM_train, 
    recommenders=[recommender_slim, recommender_p3a, hybrid_similarity], 
    cutoffs=[500, 100, 10]
)

result_df, _ = evaluator.evaluateRecommender(pipeline_recommender)
display(result_df)

Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 10456 (100.0%) in 35.77 sec. Users per second: 292


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.095285,0.160154,0.133994,0.049717,0.084183,0.273494,0.150159,0.111372,0.511477,0.363751,...,0.802826,0.410627,0.802826,0.109095,11.059248,0.998868,0.314784,0.851961,1.253873,0.315156


In [25]:
pipeline_recommender = PipelineRecommender(
    URM_train, 
    recommenders=[recommender_slim, hybrid_similarity, recommender_p3a], 
    cutoffs=[500, 100, 10]
)

result_df, _ = evaluator.evaluateRecommender(pipeline_recommender)
display(result_df)

Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 10456 (100.0%) in 35.58 sec. Users per second: 294


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.092626,0.156961,0.131649,0.047833,0.081607,0.268045,0.146473,0.108743,0.508512,0.353281,...,0.802826,0.408246,0.802826,0.094135,10.915345,0.998808,0.271616,0.840876,1.318638,0.311069


In [26]:
pipeline_recommender = PipelineRecommender(
    URM_train, 
    recommenders=[hybrid_similarity, recommender_p3a, recommender_slim], 
    cutoffs=[500, 100, 10]
)

result_df, _ = evaluator.evaluateRecommender(pipeline_recommender)
display(result_df)

Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 10456 (100.0%) in 30.84 sec. Users per second: 339


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.096729,0.161391,0.134808,0.051052,0.085452,0.281794,0.152703,0.112637,0.520371,0.374193,...,0.802826,0.417767,0.802826,0.085375,10.948049,0.999059,0.246342,0.843395,1.234038,0.312293


In [27]:
pipeline_recommender = PipelineRecommender(
    URM_train, 
    recommenders=[hybrid_similarity, recommender_slim, recommender_p3a], 
    cutoffs=[500, 100, 10]
)

result_df, _ = evaluator.evaluateRecommender(pipeline_recommender)
display(result_df)

Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 10456 (100.0%) in 30.81 sec. Users per second: 339


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.092636,0.156526,0.131151,0.047868,0.081592,0.268053,0.146378,0.108579,0.507938,0.353398,...,0.802826,0.407786,0.802826,0.081093,10.747179,0.998698,0.233986,0.827921,1.315881,0.311402


In [28]:
pipeline_recommender = PipelineRecommender(
    URM_train,
    recommenders=[hybrid_similarity, recommender_slim], 
    cutoffs=[100, 10]
)

result_df, _ = evaluator.evaluateRecommender(pipeline_recommender)
display(result_df)

Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 10456 (100.0%) in 23.21 sec. Users per second: 450


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.097676,0.164717,0.138078,0.0513,0.086171,0.282314,0.154193,0.114415,0.524005,0.375518,...,0.802826,0.420685,0.802826,0.077172,10.790301,0.998918,0.222673,0.831243,1.391342,0.306537


In [29]:
def objective(trial):
    # cutoff_p3alpha = trial.suggest_int('cutoff_p3alpha', 11, 2000)
    # cutoff_merging = trial.suggest_int('cutoff_merging', 10, cutoff_p3alpha)
    cutoff = trial.suggest_int('cutoff', 10, 2000)
    
    pipeline_recommender = PipelineRecommender(
        URM_train, 
        recommenders=[hybrid_similarity, recommender_slim], 
        cutoffs=[cutoff, 10]
    )

    result_df, _ = evaluator.evaluateRecommender(pipeline_recommender)
    return result_df.loc[10, 'MAP']

In [30]:
try:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/tuning_hybrid.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

In [31]:
if config['tune_parameters']:
    study = optuna.create_study(direction='maximize', study_name='hyperparameters_tuning_2stage_pipeline', 
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)
    study.optimize(objective, n_trials=400)

[I 2023-12-04 16:26:04,533] A new study created in RDB with name: hyperparameters_tuning_2stage_pipeline


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 10456 (100.0%) in 23.56 sec. Users per second: 444


[I 2023-12-04 16:26:28,292] Trial 0 finished with value: 0.05123705839132441 and parameters: {'cutoff': 751}. Best is trial 0 with value: 0.05123705839132441.


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 10456 (100.0%) in 23.92 sec. Users per second: 437


[I 2023-12-04 16:26:52,343] Trial 1 finished with value: 0.05123678134222372 and parameters: {'cutoff': 1894}. Best is trial 0 with value: 0.05123705839132441.


...

[I 2023-12-04 19:01:54,070] Trial 397 finished with value: 0.051270228379544106 and parameters: {'cutoff': 160}. Best is trial 11 with value: 0.051586052968752405.


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 10456 (100.0%) in 23.52 sec. Users per second: 445


[I 2023-12-04 19:02:17,773] Trial 398 finished with value: 0.05129662015277978 and parameters: {'cutoff': 89}. Best is trial 11 with value: 0.051586052968752405.


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 10456 (100.0%) in 23.32 sec. Users per second: 448


[I 2023-12-04 19:02:41,268] Trial 399 finished with value: 0.05140482490496884 and parameters: {'cutoff': 45}. Best is trial 11 with value: 0.051586052968752405.


In [32]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

In [33]:
# if config['tune_parameters']:
#     fig = optuna.visualization.plot_param_importances(study)
#     fig.show()

In [34]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_slice(study)
    fig.show()

In [35]:
if config['tune_parameters']:
    with open('/kaggle/working/best_params_2stage_pipeline.json', 'w') as params_file:
        json.dump(study.best_params, params_file)
        
    if config['save_github']: 
        upload_file(
            '/kaggle/working/best_params_2stage_pipeline.json', 
            'tuning_results/best_params_2stage_pipeline.json', 
            'Hybrid Pipeline tuning results (from kaggle notebook)'
        )
    
elif config['copy_prev_best_params']:
    shutil.copyfile(
        '/kaggle/input/Recsys_Challenge_2023/tuning_results/best_params_2stage_pipeline.json', 
        '/kaggle/working/best_params_2stage_pipeline.json'
    )

In [36]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        'tuning_results/tuning_hybrid.db', 
        'Pipeline hybrid recommender tuning db updated results (from kaggle notebook)'
    )

# Training best model

In [37]:
recommender_item = ItemKNNCFRecommender(URM_all)
recommender_item.fit(**params_item_knn)

recommender_rp3 = RP3betaRecommender(URM_all)
recommender_rp3.fit(**params_rp3)

recommender_p3a = P3alphaRecommender(URM_all)
recommender_p3a.fit(**params_p3alpha)

recommender_slim = SLIMElasticNetRecommender(URM_all)
recommender_slim.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'slim_elastic_all.zip')

recommender_ials = IALSRecommender(URM_all)
recommender_ials.load_model('/kaggle/working/', 'ials_all.zip')

hybrid_similarity = ItemKNNSimilarityHybridRecommender(URM_all, recommender_item.W_sparse, recommender_rp3.W_sparse)
hybrid_similarity.fit(**params_merging)

ItemKNNCFRecommender: URM Detected 386 ( 3.0%) users with no interactions.
ItemKNNCFRecommender: URM Detected 125 ( 0.6%) items with no interactions.
Similarity column 22347 (100.0%), 3887.66 column/sec. Elapsed time 5.75 sec
RP3betaRecommender: URM Detected 386 ( 3.0%) users with no interactions.
RP3betaRecommender: URM Detected 125 ( 0.6%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 2131.86 column/sec. Elapsed time 10.48 sec
P3alphaRecommender: URM Detected 386 ( 3.0%) users with no interactions.
P3alphaRecommender: URM Detected 125 ( 0.6%) items with no interactions.
P3alphaRecommender: Similarity column 22347 (100.0%), 2409.98 column/sec. Elapsed time 9.27 sec
SLIMElasticNetRecommender: URM Detected 386 ( 3.0%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 125 ( 0.6%) items with no interactions.
SLIMElasticNetRecommender: Loading model from file '/kaggle/working/Recsys_Challenge_2023/models/slim_elastic_all.zip'
SLIMElasti

In [38]:
with open('/kaggle/working/best_params_2stage_pipeline.json', 'r') as params_file:
    params = json.load(params_file)
    
pipeline_recommender = PipelineRecommender(
    URM_all, 
    recommenders=[hybrid_similarity, recommender_slim],
    cutoffs=[params['cutoff'], 10]
)

Recommender_Base_Class: URM Detected 386 ( 3.0%) users with no interactions.
Recommender_Base_Class: URM Detected 125 ( 0.6%) items with no interactions.


In [39]:
recommendations = pipeline_recommender.recommend(user_ids_test['user_id'].unique() - 1, cutoff=10, return_scores=False)
recommendations = [' '.join([str(item + 1) for item in r]).strip() for r in recommendations]

submission_df = pd.DataFrame({'user_id': user_ids_test['user_id'].unique(), 'item_list': recommendations})
display(submission_df.head())
submission_df.to_csv('/kaggle/working/submission.csv', index=False)

,user_id,item_list
0,1,101 36 123 506 515 403 694 1546 1422 592
1,2,1095 12 47 1522 50 656 949 359 196 102
2,3,59 857 2172 1281 2748 536 259 584 970 239
3,4,249 28 50 314 146 139 171 5 254 7
4,5,1570 170 77 95 471 131 5138 135 116 7033
